# Import the necessary libraries

In [8]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
from load_data import Load_Data
from fill_gender_based_on_title import Fill_gender_based_on_title
from data_processor import DataProcessor
from data_processor import HypothesisTester
from data_processor import ReportGenerator



# Load data

In [5]:
# Create an instance of CSVReader
csv_reader = Load_Data('../data/cleaned_data.csv')

# Load the data
csv_reader.load_data()

# Get the loaded data
df = csv_reader.get_data()

# Now, you can use raw_data as needed

c:\Users\wubeshet.abera\Projects\Ten-Academy\updated-insurance\v3\AlphaCare-insurance-predictive-analysis\scripts\load_data.py:11: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(self.file_path)


Data successfully loaded from ../data/cleaned_data.csv


# Fill gender columns as male and female

In [6]:
df = Fill_gender_based_on_title(df)

# Remove duplicates from the 'Province' column
df_unique = df.drop_duplicates(subset='Province')

# Print unique values in the 'Province' column
unique_provinces = df_unique['Province'].unique()
print(unique_provinces)

Gender
Male      934365
Female     65733
Name: count, dtype: int64
['Gauteng' 'KwaZulu-Natal' 'Mpumalanga' 'Eastern Cape' 'Western Cape'
 'Limpopo' 'North West' 'Free State' 'Northern Cape']


# A/B testing

In [9]:
# Initialize DataProcessor
data_processor = DataProcessor(df)

# Select KPI (e.g., 'TotalClaims' or 'TotalPremium')
kpi = 'TotalClaims'  # You only need to pass the column name

# Segment data by multiple provinces (for testing risk differences across more than two provinces)
group_conditions = {
    'Gauteng': (df['Province'] == 'Gauteng'),
    'KwaZulu-Natal': (df['Province'] == 'KwaZulu-Natal'),
    'Western Cape': (df['Province'] == 'Western Cape'),  # Add as many provinces as needed
    'Mpumalanga': (df['Province'] == 'Mpumalanga'),  # Add as many provinces as needed
    'Limpopo': (df['Province'] == 'Limpopo'),  # Add as many provinces as needed
    'North West': (df['Province'] == 'North West'),  # Add as many provinces as needed
    'Free State': (df['Province'] == 'Free State'),  # Add as many provinces as needed
    'Northern Cape': (df['Province'] == 'Northern Cape'),  # Add as many provinces as needed
    'Eastern Cape': (df['Province'] == 'Eastern Cape')  # Add as many provinces as needed
}

# Segment data into multiple groups based on the provinces
groups = data_processor.segment_data('Province', group_conditions)

# Initialize the HypothesisTester
tester = HypothesisTester()

# Perform an ANOVA test between the groups on 'TotalClaims'
p_value = tester.anova_test(groups, kpi)

# Analyze the results
result = tester.analyze_results(p_value)
tester.add_result('Risk difference across provinces', p_value, result)

# Generate the report
report_generator = ReportGenerator(tester.results)
report = report_generator.generate_report()
print(report)



A/B Hypothesis Testing Report
Hypothesis: Risk difference across provinces
P-Value: 1.6782057588675903e-07
Result: Reject the null hypothesis
----------------------------------------

